In [ ]:
# from PyPDF2 import PdfReader
# from collections import Counter
# import re
# import os
# import pandas as pd

# def extract_sections(pdf_path, sections_of_interest):
#     """
#     Extract specific sections (e.g., Abstract, Introduction) from a PDF.

#     Args:
#         pdf_path (str): Path to the PDF file.
#         sections_of_interest (list of str): List of section titles to extract.

#     Returns:
#         dict: A dictionary with section titles as keys and their content as values.
#     """
#     reader = PdfReader(pdf_path)
#     section_texts = {section: "" for section in sections_of_interest}

#     current_section = None
#     for page in reader.pages:
#         text = page.extract_text()
#         lines = text.split("\n")
#         for line in lines:
#             line_clean = line.strip()
#             if line_clean in sections_of_interest:
#                 current_section = line_clean
#             elif current_section and line_clean:
#                 section_texts[current_section] += " " + line_clean

#     return section_texts

# def count_keywords(section_texts, keywords):
#     """
#     Count occurrences of keywords in specified sections.

#     Args:
#         section_texts (dict): Dictionary with section titles as keys and their content as values.
#         keywords (list of str): List of keywords to search for.

#     Returns:
#         dict: A dictionary with keywords as keys and their counts as values.
#     """
#     keyword_counts = {key: 0 for key in keywords}
#     for section, text in section_texts.items():
#         word_counter = Counter(text.lower().split())  # Use a Counter to avoid overcounting phrases
#         for keyword in keywords:
#             keyword_counts[keyword] += word_counter[keyword.lower()]

#     return keyword_counts

# def find_sentences_with_keyword(section_texts, keyword):
#     """
#     Find sentences containing a specific keyword in the sections.

#     Args:
#         section_texts (dict): Dictionary with section titles as keys and their content as values.
#         keyword (str): Keyword to search for.

#     Returns:
#         list: List of sentences containing the keyword.
#     """
#     sentences_with_keyword = []
#     for section, text in section_texts.items():
#         sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
#         for sentence in sentences:
#             if keyword.lower() in sentence.lower():
#                 sentences_with_keyword.append(sentence.strip())
#     return sentences_with_keyword

# def process_papers(folder_path, sections_of_interest, keywords):
#     """
#     Process all papers in a folder, count keywords, and compile results into a DataFrame.

#     Args:
#         folder_path (str): Path to the folder containing PDF files.
#         sections_of_interest (list of str): List of section titles to extract.
#         keywords (list of str): List of keywords to search for.

#     Returns:
#         pd.DataFrame: DataFrame containing paper names and keyword counts.
#     """
#     results = []

#     for filename in os.listdir(folder_path):
#         if filename.endswith(".pdf"):
#             pdf_path = os.path.join(folder_path, filename)
#             section_texts = extract_sections(pdf_path, sections_of_interest)
#             keyword_counts = count_keywords(section_texts, keywords)
#             results.append([filename] + [keyword_counts.get(keyword, 0) for keyword in keywords])

#     columns = ["Paper Name"] + keywords
#     return pd.DataFrame(results, columns=columns)

# if __name__ == "__main__":
#     # Folder containing PDF files
#     #folder_path = "Paper Neurips"

#     # Define sections of interest
#     sections_of_interest = ["Abstract", "Introduction", "Discussion", "Conclusion"]

#     # Define keywords to search for
#     keywords = [
#         "Classification", "Prediction", "Generation", "Understanding", "Defenses", "Transparency", "Novelty", "Simplicity", "Generalization", "Flexibility/Extensibility", "Robustness", 
#         "Realistic output", "Formal description/analysis", "Theoretical guarantees", "Approximation", "Quantitative evidence (e.g. experiments)", "Qualitative evidence (e.g. examples)", 
#         "Scientific methodology", "Controllability (of model owner)", "Human-like mechanism", "Low cost", "Large scale", "Promising", "Generality", "Principled", "Exactness", "Preciseness", 
#         "Concreteness", "Automatic", "Performance", "Accuracy", "Avoiding train/test discrepancy", "State-of-the-art", "Efficiency", "Reduced training time", "Memory efficiency", 
#         "Data efficiency", "Label efficiency", "Energy efficiency", "Effectiveness", "Successful", "Building on classic work", "Building on recent work", "Unifying ideas", 
#         "integrating components", "Identifying limitations", "Critique", "Understanding", "Improvement", "Progress", "Used in practice", "Reproducibility", "Easy to implement", 
#         "Requires few resources", "Parallelizability", "distributed", "Facilitating use", "Scales up", "Applies to real world", "Learning from humans", "Practical", "Useful", "Interpretable", 
#         "Transparent", "Privacy", "Fairness", "Not socially biased", "User influence", "Collective influence", "Deferral to humans", "Critiqability", "Beneficence", "Non-maleficence", "Justice", 
#         "Respect for Persons", "Autonomy", "Explicability", "Respect for Law and public interest", "Security", "Easy to work with", "Realistic world model", "Fast"
#     ]

#     # Process all papers and create a DataFrame
#     keyword_matrix_neurips = process_papers("Paper Neurips", sections_of_interest, keywords)
#     keyword_matrix_icml = process_papers("Paper ICML", sections_of_interest, keywords)

#     # Save the results to a CSV file
#     keyword_matrix_neurips.to_csv("keyword_counts_neurips.csv", index=False)
#     print(f"Keyword counts saved to keyword_counts_neurips.csv")

#     keyword_matrix_icml.to_csv("keyword_counts_icml.csv", index=False)
#     print(f"Keyword counts saved to keyword_counts_icml.csv")

#     # Search for sentences containing a specific keyword
#     # keyword_to_search = input("Enter a keyword to search for sentences: ")
#     # for filename in os.listdir(folder_path):
#     #     if filename.endswith(".pdf"):
#     #         pdf_path = os.path.join(folder_path, filename)
#     #         section_texts = extract_sections(pdf_path, sections_of_interest)
#     #         sentences = find_sentences_with_keyword(section_texts, keyword_to_search)
#     #         if sentences:
#     #             print(f"\nSentences containing '{keyword_to_search}' in {filename}:")
#     #             for sentence in sentences:
#     #                 print(f"- {sentence}")
